In [1]:
import os 
import pandas as pd

### Build Reference Fasta

In [2]:
orig_seqs = pd.read_excel('../05052016_sanger_validation/input/Yeast ORFeome HIP ORFeome collection_v3 Details inc 384-format.xlsx')

seq_source = {str(seq_name): str(seq) for seq_name, seq in zip(orig_seqs['ORF_NAME'], orig_seqs['SEQ']) if str(seq_name) != 'nan'}

with open('./temp/ref.fasta','w') as handle:
    for seq in seq_source:
        handle.write('>{}\n{}\n'.format(seq, seq_source[seq]))

### Build query Fasta

In [3]:
all_quey_names = []
qw_file = open('./temp/query.fasta', 'w')

for file in os.listdir('./input/'):
    if file[-4:] != '.seq':
        continue
    name = file.replace('--', '-') 
    name = '_'.join(name.split('_')[0].split('-')).upper()
    all_quey_names.append(name)
    with open('./input/{}'.format(file), 'r') as handle:
        handle.__next__()
        print('>{}\n{}'.format(name, handle.__next__()), file=qw_file)
qw_file.close()

### Blasting

In [4]:
os.system('blastn -query ./temp/query.fasta -subject ./temp/ref.fasta -outfmt 6 -out ./temp/blastresults.txt')

blast_hits = {}
with open('./temp/blastresults.txt', 'r') as handle:
    for line in handle:
        line = line.split()
        blast_hits[line[0]] = blast_hits.get(line[0], []) + [line[1]]


### Check matches

In [5]:
# Fetch reference file
source_dict = {}

my_table = pd.read_table('../output/ORF_selection_20160519/DB_barcodeORFeome_20160519.tsv')
for xloc, gene in zip(my_table['Location'], my_table['ORF']):
    a = xloc.split('_')[0][2:]
    b = xloc.split('_')[1]
    built_name = 'DB_{}_{}'.format(a,b)
    source_dict[built_name] = gene
    
my_table = pd.read_table('../output/ORF_selection_20160527/AD_barcodeORFeome_20160527.tsv')
for xloc, gene in zip(my_table['Location'], my_table['ORF']):
    a = xloc.split('_')[0][2:]
    b = xloc.split('_')[1]
    built_name = 'AD_{}_{}'.format(a,b)
    source_dict[built_name] = gene


In [6]:
c_wr, c_ok , c_mm = 0, 0, 0

for query in sorted(all_quey_names):
    if query not in blast_hits:
        print('WARNING: No blast hits for colony "{}"'.format(query))
        c_wr += 1 
    elif source_dict[query] in blast_hits[query]:
        print('OK: Gene "{}" is in spot {}'.format(source_dict[query], query))
        c_ok += 1
    else:
        print('MISSMATCH: Spot "{}" differs, expected "{}" got "{}"'.format(query, source_dict[query], blast_hits[query]))
        c_mm += 1

print('\nSummary:')
print('OK hits:', c_ok)
print('No hits/bad quality:', c_wr)
print('Wrong hits: ', c_mm)

OK: Gene "YMR152W" is in spot AD_2_B8
OK: Gene "YFL026W" is in spot AD_2_H10
OK: Gene "YLR307C-A" is in spot AD_4_D1
OK: Gene "YGR059W" is in spot AD_4_K14
OK: Gene "YDL035C" is in spot AD_4_P24
OK: Gene "YIL145C" is in spot AD_5_D2
OK: Gene "YHR025W" is in spot AD_5_F16
OK: Gene "YDR118W" is in spot AD_6_D3
OK: Gene "YFL037W" is in spot AD_6_G14
OK: Gene "YDR135C" is in spot AD_6_I10
OK: Gene "YLR099W-A" is in spot AD_6_O10
OK: Gene "YHL040C" is in spot AD_6_P1
OK: Gene "YER026C" is in spot AD_7_A1
OK: Gene "YAR023C" is in spot AD_7_C19
OK: Gene "YLR350W" is in spot AD_7_F10
OK: Gene "YGL037C" is in spot AD_7_H3
MISSMATCH: Spot "DB_1_A1" differs, expected "YGR122W" got "['YLR378C']"
OK: Gene "YDR476C" is in spot DB_1_D20
OK: Gene "YDR320C-A" is in spot DB_1_K24
OK: Gene "YHR117W" is in spot DB_1_P2
OK: Gene "YDR536W" is in spot DB_2_B12
OK: Gene "YOR069W" is in spot DB_2_G2
OK: Gene "YGR293C" is in spot DB_2_J1
OK: Gene "YIL046W-A" is in spot DB_2_M24
OK: Gene "YPL015C" is in spot DB_